In [2]:
import time#
from pprint import pprint#

import dotenv#
from pathlib import Path#
import os#
dotenv.load_dotenv(verbose=True)

questdb_config = {
    "host" : os.getenv('questdb_host'),
    "influxdb_port" : os.getenv('questdb_influxdb_port'),
    "postgres_port" : os.getenv('questdb_postgres_port'),
    "health_port" :   os.getenv('questdb_health_port'),
}

print(questdb_config)



{'host': 'localhost', 'influxdb_port': '9009', 'postgres_port': '8812', 'health_port': '9003'}


In [3]:
#check alive
import urllib.request as urlreq#
# import urllib3.request as urlreq#
health_url = f"http://{questdb_config['host']}:{questdb_config['health_port']}/status"
f = urlreq.urlopen(health_url)
if f.read().decode().index('Healthy') > 0:
    print("Status: Healthy")


Status: Healthy


In [6]:
#questdb write with influxdb line protocol
from questdb.ingress import Sender, IngressError
import sys
import datetime

table_name = 'candles_btc-usdt'
datas = [
    [1630425660000, 48772.2, 48801.4, 48714.6, 48735.5, 1009.0],
]

try:
    with Sender(questdb_config['host'], questdb_config['influxdb_port']) as sender:
        # Record with provided designated timestamp (using the 'at' param)
        # Notice the designated timestamp is expected in Nanoseconds,
        # but timestamps in other columns are expected in Microseconds. 
        # The API provides convenient functions
        for row in datas:
            
            ts = datetime.datetime.utcfromtimestamp(row[0]/1000);
            print(ts)
            sender.row(
                table_name,
                columns={
                    # 'timestamp': row[0]*1000,
                    # 'timestamp': ts,
                    'open':row[1],
                    'high':row[2],
                    'low':row[3],
                    'close':row[4],
                    'volume':row[5],
                },
            # at=datetime.datetime.utcnow())
                at=ts
            )
        #END for

        # We recommend flushing periodically, for example every few seconds.
        # If you don't flush explicitly, the client will flush automatically
        # once the buffer is reaches 63KiB and just before the connection
        # is closed.
        sender.flush()

except IngressError as e:
    sys.stderr.write(f'Got error: {e}\n')







2021-08-31 16:01:00
